# Introduction

### I would like to solve the problem of finding the popular libraries in the state of Kansas so that user residing in state of Kansas can find libraries close to their location and have access to free books in the library

# Data Section

### We will start with use of different steps in data science methodology. Now that the business requirement is already laid, it is time to find the approach we are going to take to collect, understand, analyse and prepare the data. The first step include collecting the data making use of publicly available library dataset, foursquare api to get the location of the library, publicly available popular books dataset like New york best sellers list. We are going to scrap away the PII data and only keep the publicly available data during the stage of data preprocessing. Now we analyse the data make sure what are the individual fields/columns are relevance and remove the unwanted fields. we will convert the text column to int by using transpose method so that we can fit model on the data. During the whole process of analyzing the data , we will make use of different plots to understand the data to its depth


### Dataset

##### 1.  Libraries in Kansas 
#### 2.  Popular books
#### 3.  Foursquare API